In [ ]:
from ShopgunAPI import ShopgunAPI
import json
import io

shopgunAPI = ShopgunAPI(verbose=True);
#shopgunAPI.getStores();

In [ ]:
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken',\
                      'coop prix','joker', 'bunnpris', 'extra', 'spar']
selectedBrandNames = ['spar']
selectedCatalogList = shopgunAPI.getCatalogs(selectedBrandNames=selectedBrandNames)

In [ ]:
from MatifyAPI import MatifyAPI
matifyAPI = MatifyAPI()
#Get offers in selected catalogs
for catalogStore, catalogId, catalogRunFrom, catalogBrand in selectedCatalogList:
    offerDescList = shopgunAPI.getOfferDescriptionsOfCatalog(catalogId, catalogStore)
    JsonFileName = shopgunAPI.generateCatalogFileName(catalogBrand, catalogId, catalogRunFrom)
    with io.open(JsonFileName, 'w', encoding="utf-8") as json_file:
        data = json.dumps(offerDescList, ensure_ascii=False)     
        json_file.write(unicode(data))
        matifyAPI.uploadCatalog(brandName=catalogBrand, catalogData=data, catalogFileName=JsonFileName)
    print "Done."

## Download deals from before

In [ ]:
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken', 
                      'coop obs', 'coop prix','joker', 'bunnpris', 'extra', 'spar']

catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95'+
                               '&r_lng=10.75&r_radius=700000&api_av=0.1.33'+
                               '&offset=10&limit=100'+
                               '&run_from=2015-01-30T23:00:00+0000'+
                               '&run_till=2015-02-13T22:59:59+0000',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from'], 
                        next(brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower())] 
                       for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From', 'Brand'])

In [ ]:
catalogsResponse

In [ ]:
catalogs = json.loads(catalogsResponse.text)
